In [211]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [212]:
#Load Dataset
dataset = pd.read_csv('https://raw.githubusercontent.com/gmbt18/CS176-Project/cedric-preproc/Exasens.csv')

#Removing unnecessary rows and cols
dataset.drop(dataset.columns[[9,10,11,12]],axis=1,inplace=True)
# dataset.drop(dataset.columns[[1,2,3,4,5]],axis=1,inplace=True)
dataset.drop([0,1],axis=0,inplace=True)

#Renaming cols
dataset.columns.values[2] = "Img Min"
dataset.columns.values[3] = "Img Avg"
dataset.columns.values[4] = "Real Min"
dataset.columns.values[5] = "Real Avg"

#Rearranging Cols
# dataset = dataset[['Gender','Age','Smoking','Diagnosis']]

#Rearranging Rows
# dataset.sort_values(by=['Diagnosis'], inplace=True)
dataset = dataset.reset_index()
del dataset['index']

In [213]:
dataset.head()

,Diagnosis,ID,Img Min,Img Avg,Real Min,Real Avg,Gender,Age,Smoking
0,COPD,301-4,-320.61,-300.5635307,-495.26,-464.1719907,1.0,77.0,2.0
1,COPD,302-3,-325.39,-314.7503595,-473.73,-469.2631404,0.0,72.0,2.0
2,COPD,303-3,-323,-317.4360556,-476.12,-471.8976667,1.0,73.0,3.0
3,COPD,304-4,-327.78,-317.3996698,-473.73,-468.856388,1.0,76.0,2.0
4,COPD,305-4,-325.39,-316.1557853,-478.52,-472.8697828,0.0,65.0,2.0


In [214]:
ds2 = dataset.iloc[:,[0,6,7,8]]

# Group Ages
ds2['Age'] = ds2['Age'].mask(ds2['Age']<40,1)
ds2['Age'] = ds2['Age'].mask((ds2['Age']>=40) & (ds2['Age']<60),2)
ds2['Age'] = ds2['Age'].mask(ds2['Age']>=60,3)

ds2['Diagnosis'] = ds2.Diagnosis.astype(str)
ds2['Age'] = ds2.Age.astype(str)
ds2['Gender'] = ds2.Gender.astype(str)
ds2['Smoking'] = ds2.Smoking.astype(str)

# ds2.replace('0.0',0,inplace=True)
# ds2.replace('1.0',0,inplace=True)

ds2['Diagnosis'] = pd.Categorical(ds2.Diagnosis)
ds2['Age'] = pd.Categorical(ds2.Age)
ds2['Gender'] = pd.Categorical(ds2.Gender)
ds2['Smoking'] = pd.Categorical(ds2.Smoking)


print(ds2.dtypes)

# ds = (dataset.iloc[:,[7,8]].groupby(['Diagnosis', 'Smoking']).count().unstack())
# print(ds.to_string())
# print(ds2.to_string())
ds2 = pd.get_dummies(ds2)

Diagnosis    category
Gender       category
Age          category
Smoking      category
dtype: object


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [215]:
print(ds2.dtypes)
frequent_itemsets = apriori(ds2, min_support=0.07, use_colnames=True)

Diagnosis_Asthma      uint8
Diagnosis_COPD        uint8
Diagnosis_HC          uint8
Diagnosis_Infected    uint8
Gender_0.0            uint8
Gender_1.0            uint8
Age_1.0               uint8
Age_2.0               uint8
Age_3.0               uint8
Smoking_1.0           uint8
Smoking_2.0           uint8
Smoking_3.0           uint8
dtype: object


In [216]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

diagnoses={'Diagnosis_COPD','Diagnosis_Infected','Diagnosis_Asthma','Diagnosis_HC'}

# for rule in rules:
#   if(len(rules['antecedents'][4].intersection(diagnoses)) != 0):
#     print('nope')

rules_summary = rules.iloc[:,[0,1,4,5,6]]

for i in range(len(rules_summary)):
  if(len(rules_summary['consequents'][i].intersection(diagnoses)) == 0
     or len(rules_summary['consequents'][i]) > 1):
    rules_summary.drop(index=i,inplace=True)
  elif(len(rules_summary['antecedents'][i].intersection(diagnoses)) != 0):
    rules_summary.drop(index=i,inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [217]:
rules_summary.sort_values(by='lift',ascending=False)

,antecedents,consequents,support,confidence,lift
148,"(Smoking_2.0, Gender_1.0, Age_3.0)",(Diagnosis_COPD),0.115288,0.807018,4.075949
54,"(Gender_1.0, Age_3.0)",(Diagnosis_COPD),0.132832,0.726027,3.666898
65,"(Smoking_2.0, Age_3.0)",(Diagnosis_COPD),0.147870,0.719512,3.633992
59,"(Smoking_2.0, Gender_1.0)",(Diagnosis_COPD),0.117794,0.602564,3.043330
11,(Age_3.0),(Diagnosis_COPD),0.182957,0.593496,2.997530
12,(Smoking_2.0),(Diagnosis_COPD),0.157895,0.420000,2.121266
9,(Gender_1.0),(Diagnosis_COPD),0.140351,0.352201,1.778839
106,"(Age_1.0, Gender_0.0)",(Diagnosis_Infected),0.082707,0.326733,1.629579
88,"(Gender_1.0, Age_1.0)",(Diagnosis_HC),0.072682,0.644444,1.607083
27,(Age_1.0),(Diagnosis_Infected),0.115288,0.315068,1.571404


In [218]:
rules_summary.sort_values(by='confidence',ascending=False)

,antecedents,consequents,support,confidence,lift
148,"(Smoking_2.0, Gender_1.0, Age_3.0)",(Diagnosis_COPD),0.115288,0.807018,4.075949
54,"(Gender_1.0, Age_3.0)",(Diagnosis_COPD),0.132832,0.726027,3.666898
65,"(Smoking_2.0, Age_3.0)",(Diagnosis_COPD),0.147870,0.719512,3.633992
88,"(Gender_1.0, Age_1.0)",(Diagnosis_HC),0.072682,0.644444,1.607083
98,"(Age_1.0, Smoking_1.0)",(Diagnosis_HC),0.145363,0.604167,1.506641
59,"(Smoking_2.0, Gender_1.0)",(Diagnosis_COPD),0.117794,0.602564,3.043330
11,(Age_3.0),(Diagnosis_COPD),0.182957,0.593496,2.997530
163,"(Age_1.0, Gender_0.0, Smoking_1.0)",(Diagnosis_HC),0.092732,0.578125,1.441699
16,(Age_1.0),(Diagnosis_HC),0.208020,0.568493,1.417680
93,"(Gender_1.0, Smoking_1.0)",(Diagnosis_HC),0.080201,0.561404,1.400000
